# Hướng dẫn truy vấn dữ liệu thị giác dùng fiftyone

Đây là hướng dẫn dùng cho các đội tham dự AI Challenge 2023. Hướng dẫn này nhằm mục đích giới thiệu cho các đội một phương pháp cơ bản để truy vấn dữ liệu dựa trên thông tin BTC cung cấp và giới thiệu công cụ fiftyone để hỗ trợ đội thi đánh giá kết quả.

## Cài đặt ban đầu

Bạn cần cài đặt môi trường để chạy được notebook này trên máy tính cá nhân của bạn. Hướng dẫn này không bao gồm phần cài đặt môi trường. Khuyến nghị: các bạn có thể cài đặt [Anaconda](https://docs.anaconda.com/free/anaconda/install/windows/).

## Cài đặt các thư viện FiftyOne và PyTorch
Hướng dẫn này dùng fiftyone là công cụ để trực quan dữ liệu và pytorch là backend chính cho các thuật toán máy học.

### Lưu ý: Đối với các bạn dùng Windows nên dùng bản fiftyone **v0.21.4**, không nên dùng bản mới nhất!

In [ ]:
! pip install fiftyone==0.21.4
! pip install torch torchvision torchaudio


Load dữ liệu keyframe từ thư mục chứa keyframe. Mỗi ảnh và thông tin đi kèm sau này sẽ được lưu trữ trong một Sample. Tất cả các Sample được lưu trong Dataset.

In [1]:
import fiftyone as fo
import fiftyone.brain as fob
import numpy as np
from glob import glob
import json
import os



Load dữ liệu keyframe từ thư mục chứa keyframe. Trong hướng dẫn này tất cả các file Keyframes_L*.zip được giải nén vào thư mục `D:\AIC\Keyframes`. Mỗi ảnh và thông tin đi kèm sau này sẽ được lưu trữ trong một `Sample`. Tất cả các `Sample` được lưu trong `Dataset`.

In [2]:
dataset = fo.Dataset.from_images_dir('D:\WorkSpace\Contest\HCM_AIC2023\dataset\keyframes', name=None, tags=None, recursive=True)

 100% |█████████████| 67619/67619 [9.3s elapsed, 0s remaining, 5.9K samples/s]       


Sau khi dữ liệu đã load lên xong. Bạn có thể truy cập vào đường vào ứng dụng web của fiftyone từ [http://localhost:5151](http://localhost:5151)

Hoặc bạn có thể chạy cell bên dưới để mở tab mới cho ứng dụng web fiftyone

### Trích xuất thêm thông tin tên của video và frameid
Thông tin `video` và `frameid` sẽ được lấy từ tên của tập tin keyframe.

In [5]:
for sample in dataset:
    _, sample['video'], sample['frameid'] = sample['filepath'][:-4].rsplit('\\', 2)
    sample.save()


Could not connect session, trying again in 10 seconds


Could not connect session, trying again in 10 seconds




Could not connect session, trying again in 10 seconds


Could not connect session, trying again in 10 seconds


Could not connect session, trying again in 10 seconds


Could not connect session, trying again in 10 seconds



Bạn có thể xem `Sample` đầu tiên của `Dataset` bằng lệnh sau:

In [6]:
print(dataset.first())

<Sample: {
    'id': '65017bf58c34b7c265c46da7',
    'media_type': 'image',
    'filepath': 'D:\\WorkSpace\\Contest\\HCM_AIC2023\\dataset\\keyframes\\L01_V001\\0001.jpg',
    'tags': [],
    'metadata': None,
    'video': 'L01_V001',
    'frameid': '0001',
}>


### Thêm thông tin kết quả của object detection.

Bước này có thể tốn của bạn nhiều thời gian để đọc hết tất cả các dữ liệu về object detection. Bạn có thể bỏ qua cell này và chạy cell này sau nếu muốn thử thêm các thông tin về vector CLIP embedding trước.

In [5]:
for sample in dataset:
    object_path = f"D:\\AIC\\objects\\{sample['video']}\\{sample['frameid']}.json"
    with open(object_path) as jsonfile:
        det_data = json.load(jsonfile)
    detections = []
    for cls, box, score in zip(det_data['detection_class_entities'], det_data['detection_boxes'], det_data['detection_scores']):
        # Convert to [top-left-x, top-left-y, width, height]
        boxf = [float(box[1]), float(box[0]), float(box[3]) - float(box[1]), float(box[2]) - float(box[0])]
        scoref = float(score)

        # Only add objects with confidence > 0.4
        if scoref > 0.4:
            detections.append(
                fo.Detection(
                    label=cls,
                    bounding_box= boxf,
                    confidence=float(score)
                )
            )
    sample["object_faster_rcnn"] = fo.Detections(detections=detections)
    sample.save()


FileNotFoundError: [Errno 2] No such file or directory: 'D:\\AIC\\objects\\L01_V001\\0001.json'

### Thêm thông tin CLIP embedding.

In [7]:
all_keyframe = glob("dataset\\keyframes\\*\\*.jpg")
video_keyframe_dict = {}
all_video = glob('dataset\\keyframes\\*')
all_video = [v.rsplit('\\',1)[-1] for v in all_video]

In [10]:
print(all_video)

['L01_V001', 'L01_V002', 'L01_V003', 'L01_V004', 'L01_V005', 'L01_V006', 'L01_V007', 'L01_V008', 'L01_V009', 'L01_V010', 'L01_V011', 'L01_V012', 'L01_V013', 'L01_V014', 'L01_V015', 'L01_V016', 'L01_V017', 'L01_V018', 'L01_V019', 'L01_V020', 'L01_V021', 'L01_V022', 'L01_V023', 'L01_V024', 'L01_V025', 'L01_V026', 'L01_V027', 'L01_V028', 'L01_V029', 'L01_V030', 'L01_V031', 'L02_V001', 'L02_V002', 'L02_V003', 'L02_V004', 'L02_V005', 'L02_V006', 'L02_V007', 'L02_V008', 'L02_V009', 'L02_V010', 'L02_V011', 'L02_V012', 'L02_V013', 'L02_V014', 'L02_V015', 'L02_V016', 'L02_V017', 'L02_V018', 'L02_V019', 'L02_V020', 'L02_V021', 'L02_V022', 'L02_V023', 'L02_V024', 'L02_V025', 'L02_V026', 'L02_V027', 'L02_V028', 'L02_V029', 'L02_V030', 'L03_V001', 'L03_V002', 'L03_V003', 'L03_V004', 'L03_V005', 'L03_V006', 'L03_V007', 'L03_V008', 'L03_V009', 'L03_V010', 'L03_V011', 'L03_V012', 'L03_V013', 'L03_V014', 'L03_V015', 'L03_V016', 'L03_V017', 'L03_V018', 'L03_V019', 'L03_V020', 'L03_V021', 'L03_V022', 'L0

Đọc thông tin clip embedding được cung cấp.

Lưu ý: Các bạn cần tải đúng bản CLIP embedding từ model **CLIP ViT-B/32**

Tạo dictionary `video_keyframe_dict` với `video_keyframe_dict[video]` thông tin danh sách `keyframe` của `video`

In [11]:
for kf in all_keyframe:
    _, vid, kf = kf[:-4].rsplit('\\',2)
    if vid not in video_keyframe_dict.keys():
        video_keyframe_dict[vid] = [kf]
    else:
        video_keyframe_dict[vid].append(kf)

Do thông tin vector CLIP embedding được cung cấp được lưu theo từng video nhầm mục đích tối ưu thời gian đọc dữ liệu. Cần sort lại danh sách `keyframe` của từng `video` để đảm bảo thứ tự đọc đúng với vector embedding được cung cấp.

In [12]:
for k,v in video_keyframe_dict.items():
    video_keyframe_dict[k] = sorted(v)

In [ ]:
video_keyframe_dict


Could not connect session, trying again in 10 seconds


Could not connect session, trying again in 10 seconds


Could not connect session, trying again in 10 seconds



Tạo dictionary `embedding_dict` với `embedding_dict[video][keyframe]` lưu thông tin vector CLIP embedding của `keyframe` trong `video` tương ứng

In [18]:
embedding_dict = {}
for v in all_video:
    clip_path = f'dataset\\clip-features-vit-b32\\{v}.npy'
    print(clip_path)
    a = np.load(clip_path)
    embedding_dict[v] = {}
    for i,k in enumerate(video_keyframe_dict[v]):
        embedding_dict[v][k] = a[i]


dataset\clip-features-vit-b32\L01_V001.npy
dataset\clip-features-vit-b32\L01_V002.npy
dataset\clip-features-vit-b32\L01_V003.npy
dataset\clip-features-vit-b32\L01_V004.npy
dataset\clip-features-vit-b32\L01_V005.npy
dataset\clip-features-vit-b32\L01_V006.npy
dataset\clip-features-vit-b32\L01_V007.npy
dataset\clip-features-vit-b32\L01_V008.npy
dataset\clip-features-vit-b32\L01_V009.npy
dataset\clip-features-vit-b32\L01_V010.npy
dataset\clip-features-vit-b32\L01_V011.npy
dataset\clip-features-vit-b32\L01_V012.npy
dataset\clip-features-vit-b32\L01_V013.npy
dataset\clip-features-vit-b32\L01_V014.npy
dataset\clip-features-vit-b32\L01_V015.npy
dataset\clip-features-vit-b32\L01_V016.npy
dataset\clip-features-vit-b32\L01_V017.npy
dataset\clip-features-vit-b32\L01_V018.npy
dataset\clip-features-vit-b32\L01_V019.npy
dataset\clip-features-vit-b32\L01_V020.npy
dataset\clip-features-vit-b32\L01_V021.npy
dataset\clip-features-vit-b32\L01_V022.npy
dataset\clip-features-vit-b32\L01_V023.npy
dataset\cli


Could not connect session, trying again in 10 seconds



Tạo danh sách `clip_embedding` ứng với danh sách `sample` trong `dataset`.

In [14]:
clip_embeddings = []
for sample in dataset:
    clip_embedding = embedding_dict[sample['video']][sample['frameid']]
    clip_embeddings.append(clip_embedding)


In [21]:
a = np.load('D:\WorkSpace\Contest\HCM_AIC2023\dataset\clip_embeddings.npy')


Could not connect session, trying again in 10 seconds



In [20]:
for sample in dataset:
    print(sample)
    break

<Sample: {
    'id': '65017bf58c34b7c265c46da7',
    'media_type': 'image',
    'filepath': 'D:\\WorkSpace\\Contest\\HCM_AIC2023\\dataset\\keyframes\\L01_V001\\0001.jpg',
    'tags': [],
    'metadata': None,
    'video': 'L01_V001',
    'frameid': '0001',
}>



Could not connect session, trying again in 10 seconds


Could not connect session, trying again in 10 seconds


Could not connect session, trying again in 10 seconds


Could not connect session, trying again in 10 seconds


Could not connect session, trying again in 10 seconds


Could not connect session, trying again in 10 seconds


Could not connect session, trying again in 10 seconds



In [15]:
np.shape(clip_embeddings)

(67619, 512)

In [ ]:
np.save("clip_embeddings.npy", clip_embeddings)

In [ ]:
import pandas as pd

lst = []
for sample in dataset:
    lst.append([sample["filepath"], sample["video"], sample["frameid"]])

df = pd.DataFrame(lst, columns = ["filepath", "video", "frameid"])


In [ ]:
fob.compute_similarity(
    dataset,
    model="clip-vit-base32-torch",      # store model's name for future use
    embeddings=clip_embeddings,          # precomputed image embeddings
    brain_key="img_sim",
)

In [ ]:
def get_frame_feature_vector(video, frameids):
    image_ids = pd.read_csv("dataset/image_ids.csv", dtype={"filepath": "string", "video": "string", "frameid": "string"})
    image_ids = list(zip(image_ids['video'], image_ids["frameid"]))
    
    img_idx = image_ids.index((video, frameids))

    image_clipfeatures = np.load("dataset/clip_embeddings.npy")

    photo_feature = image_clipfeatures[img_idx].astype(np.float32)

    photo_feature = np.expand_dims(photo_feature, axis=0)
    return photo_feature

In [4]:
import faiss
import clip
import torch

In [5]:
device = "cuda" if torch.cuda.is_available() else "cpu"
model, preprocess = clip.load("ViT-B/32", device=device)

In [6]:
def create_index_vector(photo_features):
    index = faiss.IndexFlatL2(512)
    fe = photo_features.reshape(photo_features.shape[0], -1).astype('float32')
    print(np.shape(fe))
    index.add(fe)
    faiss.write_index(index, 'faiss_index.bin')

In [7]:
image_features = np.load("dataset/clip_embeddings.npy")
create_index_vector(image_features)

(67619, 512)


In [8]:
def get_feature_vector(text_query):
    text = clip.tokenize([text_query]).to(device)  
    text_features = model.encode_text(text).cpu().detach().numpy().astype(np.float32)
    print(text_features.shape)
    return text_features

In [48]:
def search_vector(query, faiss_index, topk):
    features_vector_search = get_feature_vector(query)
    f_dist, f_ids = faiss_index.search(features_vector_search, topk)
    faiss_index.reset()
    f_dist = np.array(f_dist[0])
    sorted_indices = np.argsort(-1 * f_dist)
    f_ids = np.array(f_ids[0])[sorted_indices]
    return f_ids

In [49]:
query = "There is a dog in a picture"
faiss_index = faiss.read_index("dataset/faiss_index.bin")
topk = 5
ids = search_vector(query, faiss_index, topk)

(1, 512)


In [50]:
ids

array([36384, 22259, 12853, 36385, 36391], dtype=int64)

In [17]:
import pandas as pd
image_ids = pd.read_csv("dataset/image_ids.csv", dtype={"filepath": "string", "video": "string", "frameid": "string"})
image_ids = list(zip(image_ids['video'], image_ids["frameid"]))

In [18]:
print(image_ids[36391])

('L06_V011', '0088')

## Từ đây các bạn có thể thử các tính năng search, filter trên ứng dụng fiftyone.

In [ ]:
# Bạn cần phải cài version umap-learn hỗ trợ.
# fob.compute_visualization(
#     dataset,
#     embeddings=clip_embeddings,
#     brain_key="img_viz"
# )
